In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Set random seed and device
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the 18 landmarks in the order they appear in your CSV
landmark_order = [
    "Nose", "Neck", "Right Shoulder", "Right Elbow", "Right Wrist",
    "Left Shoulder", "Left Elbow", "Left Wrist", "Right Hip", "Right Knee",
    "Right Ankle", "Left Hip", "Left Knee", "Left Ankle", "Right Eye",
    "Left Eye", "Right Ear", "Left Ear"
]

# Spatial graph edges for 18 joints (same as before, adjusted for index mapping)
edges = [
    (0, 1),   # Nose to Neck
    (1, 2),   # Neck to Right Shoulder
    (2, 3),   # Right Shoulder to Right Elbow
    (3, 4),   # Right Elbow to Right Wrist
    (1, 5),   # Neck to Left Shoulder
    (5, 6),   # Left Shoulder to Left Elbow
    (6, 7),   # Left Elbow to Left Wrist
    (1, 8),   # Neck to Right Hip
    (8, 9),   # Right Hip to Right Knee
    (9, 10),  # Right Knee to Right Ankle
    (1, 11),  # Neck to Left Hip
    (11, 12), # Left Hip to Left Knee
    (12, 13), # Left Knee to Left Ankle
    (0, 14),  # Nose to Right Eye
    (0, 15),  # Nose to Left Eye
    (14, 16), # Right Eye to Right Ear
    (15, 17)  # Left Eye to Left Ear
]

# Function to process a single CSV file into a skeleton sequence
def process_csv_to_skeleton(csv_path):
    # Read CSV
    df = pd.read_csv(csv_path)
    
    # Ensure all 30 frames and 18 landmarks are present
    skeleton_sequence = np.zeros((30, 18, 3))  # Shape: (frames, joints, coords)
    
    # Group by frame
    for frame in range(30):
        frame_data = df[df['frame'] == frame]
        if frame_data.empty:
            continue  # Leave as zeros if frame is missing
        
        # Map landmarks to the correct order
        for i, landmark in enumerate(landmark_order):
            landmark_data = frame_data[frame_data['landmark'] == landmark]
            if not landmark_data.empty:
                skeleton_sequence[frame, i] = landmark_data[['x', 'y', 'z']].values[0]
            # If landmark is missing, it remains zero
    
    assert skeleton_sequence.shape == (30, 18, 3), f"Unexpected shape: {skeleton_sequence.shape} for {csv_path}"
    return skeleton_sequence

# Dataset Class
class FallDataset(Dataset):
    def __init__(self, fall_dir, no_fall_dir):
        self.data = []
        self.labels = []
        
        # Process FallDataset (label = 1)
        for root, _, files in os.walk(fall_dir):
            for csv_file in files:
                if csv_file.endswith(".csv"):
                    csv_path = os.path.join(root, csv_file)
                    skeleton = process_csv_to_skeleton(csv_path)
                    self.data.append(skeleton)
                    self.labels.append(1)
        
        # Process NoFallDataset (label = 0)
        for root, _, files in os.walk(no_fall_dir):
            for csv_file in files:
                if csv_file.endswith(".csv"):
                    csv_path = os.path.join(root, csv_file)
                    skeleton = process_csv_to_skeleton(csv_path)
                    self.data.append(skeleton)
                    self.labels.append(0)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        skeleton = torch.tensor(self.data[idx], dtype=torch.float32).permute(2, 0, 1)  # (C, T, V) = (3, 30, 18)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return skeleton, label

# ST-GCN Model (unchanged)
class STGCN(nn.Module):
    def __init__(self, in_channels=3, num_joints=18, num_classes=2, graph_edges=None):
        super(STGCN, self).__init__()
        self.graph_edges = graph_edges
        self.num_joints = num_joints
        
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 1))
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding=(1, 0))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(1, 1))
        self.conv4 = nn.Conv2d(128, 128, kernel_size=(3, 1), padding=(1, 0))
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.AdaptiveAvgPool2d((1, num_joints))
        self.fc = nn.Linear(128 * num_joints, num_classes)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Training and Evaluation Functions (unchanged)
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Main Execution
if __name__ == "__main__":
    # Update these paths to your actual CSV directories
    fall_dir = "Dataset_csv/Fall"
    no_fall_dir = "Dataset_csv/NoFall"
    
    # Create dataset and split
    dataset = FallDataset(fall_dir, no_fall_dir)
    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    test_dataset = torch.utils.data.Subset(dataset, test_idx)
    
    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Initialize model, loss, and optimizer
    model = STGCN(in_channels=3, num_joints=18, num_classes=2, graph_edges=edges).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train and evaluate
    train_model(model, train_loader, criterion, optimizer, num_epochs=10)
    evaluate_model(model, test_loader)
    
    # Save the model
    torch.save(model.state_dict(), "Models/stgcn_fall_detection.pth")

Epoch 1/10, Loss: 0.6456
Epoch 2/10, Loss: 0.4015
Epoch 3/10, Loss: 0.2219
Epoch 4/10, Loss: 0.1823
Epoch 5/10, Loss: 0.1517
Epoch 6/10, Loss: 0.1225
Epoch 7/10, Loss: 0.1139
Epoch 8/10, Loss: 0.1191
Epoch 9/10, Loss: 0.1327
Epoch 10/10, Loss: 0.1068
Test Accuracy: 93.55%
